# Olympic Dataset

In [1]:
import pandas as pd
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings('ignore')

In [2]:
events = pd.read_csv('athlete_events.csv')
events.head(2)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN


In [3]:
noc = pd.read_csv('noc_regions.csv')
noc.head(2)

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles


### Lets combine the dataframes

In [236]:
data = pd.merge(events, noc, on='NOC', how='left')
data.head(1)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China,NaN


### 205 countries have participated in the Olympics

In [86]:
region = data['region'].value_counts()
len(region)

205

In [87]:
region = region.sort_values()[::-1][:50]

In [88]:
plotly.offline.iplot({
    "data": [go.Bar(x=region.values, y=region.index, orientation = 'h')],
    "layout": go.Layout(title="Number of participants per country"
                           ,width=1000,height=1000,)
})

### USA has sent the most number of athletes to the olympics followed by Germany 

In [49]:
season = data['Season'].value_counts()

In [52]:
plotly.offline.iplot({
    "data": [go.Pie(labels=season.index, values=season.values)],
})

In [107]:
sport = data['Sport'].value_counts()
sport = sport.sort_values()[::-1][:40]
plotly.offline.iplot({
    "data": [go.Bar(x=sport.values, y=sport.index, orientation = 'h', marker = dict(color = 'rgba(222,45,38,0.8)'))],
    "layout": go.Layout(title="Sports with most events"
                           ,width=700,height=800,margin=go.Margin(
        l=300,
        r=100,
        b=100,
        t=100,
        pad=4
    ),)
})

## India's Medal Count and distribution

In [87]:
India = data[(data['region']=='India')]
medals = India['Medal'].value_counts()
medals

Gold      138
Bronze     40
Silver     19
Name: Medal, dtype: int64

In [61]:
plotly.offline.iplot({
    "data": [go.Pie(labels=medals.index, values=medals.values)],
    
})

In [88]:
India['Gold'] = India['Medal'].map({'Gold': 1, 'Bronze': 0,'Silver':0})
India['Silver'] = India['Medal'].map({'Gold': 0, 'Bronze': 0,'Silver':1})
India['Bronze'] = India['Medal'].map({'Gold': 0, 'Bronze': 1,'Silver':0})
total_medals =India.groupby(['Year']).sum()

In [95]:
total_medals= total_medals.fillna(0)
trace0 = go.Scatter(
    x = total_medals.index,
    y = total_medals['Gold'],
    mode = 'lines',
    name = 'GOLD'
)
trace1 = go.Scatter(
    x = total_medals.index,
    y = total_medals['Silver'],
    mode = 'lines',
    name = 'SILVER'
)
trace2 = go.Scatter(
    x = total_medals.index,
    y = total_medals['Bronze'],
    mode = 'lines',
    name = 'BRONZE'
)

data = [trace0, trace1, trace2]

plotly.offline.iplot({
    "data": data
})

### The distribution is very lean, let's try USA

In [103]:
data = pd.merge(events, noc, on='NOC', how='left')
USA = data[(data['region']=='USA')]
medals = USA['Medal'].value_counts()
print(medals)
USA['Gold'] = USA['Medal'].map({'Gold': 1, 'Bronze': 0,'Silver':0})
USA['Silver'] = USA['Medal'].map({'Gold': 0, 'Bronze': 0,'Silver':1})
USA['Bronze'] = USA['Medal'].map({'Gold': 0, 'Bronze': 1,'Silver':0})
total_medals =USA.groupby(['Year']).sum()

Gold      2638
Silver    1641
Bronze    1358
Name: Medal, dtype: int64


In [104]:
total_medals= total_medals.fillna(0)
trace0 = go.Scatter(
    x = total_medals.index,
    y = total_medals['Gold'],
    mode = 'lines',
    name = 'GOLD'
)
trace1 = go.Scatter(
    x = total_medals.index,
    y = total_medals['Silver'],
    mode = 'lines',
    name = 'SILVER'
)
trace2 = go.Scatter(
    x = total_medals.index,
    y = total_medals['Bronze'],
    mode = 'lines',
    name = 'BRONZE'
)

data = [trace0, trace1, trace2]

plotly.offline.iplot({
    "data": data
})

In [156]:
data = pd.merge(events, noc, on='NOC', how='left')
mean = data.groupby('region').mean()
sort = mean.sort_values('Height')[::-1]

In [39]:
sort = sort.head(50)

In [40]:
trace1 = go.Bar(
    x= sort.index,
    y=sort['Weight'],
    name='Average Weight'
)
trace2 = go.Bar(
    x= sort.index,
    y= sort['Height'],
    name='Average Height'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='stack'
)


plotly.offline.iplot({
    "data": data
})

### Almost all athletes lie in the same height, weight range

In [160]:
data = pd.merge(events, noc, on='NOC', how='left')
mean = data.groupby('region').mean()
sort = mean.sort_values('Height')[::-1]
sort.tail(1)

,ID,Age,Height,Weight,Year
region,,,,,
Micronesia,60748.307692,23.846154,161.24,60.84,2008.153846


- Micronesia has the Shortest Atheletes
- Croatia has the tallest athletes

In [48]:
sort.sort_values('Weight').head(1)

,ID,Age,Height,Weight,Year
region,,,,,
Maldives,63310.571429,20.918367,163.761905,54.97619,1998.938776


In [49]:
sort.sort_values('Weight').tail(1)

,ID,Age,Height,Weight,Year
region,,,,,
Nauru,65194.923077,23.461538,167.181818,95.846154,2004.923077


- Nauru has the heaviest athletes
- Maldives has the lightest athletes


In [53]:
sort.sort_values('Age')['Age'].mean()

24.6051711301777

## Average Age of Olympic atheltes is about 25

In [237]:
data['region'] = data['region'].fillna(data['Team'])
allmedal = data.fillna(0)
allmedal['Medal'] = allmedal['Medal'].map({'Gold': 1, 'Bronze': 1,'Silver':1})

In [238]:
almedal = allmedal.groupby('region').sum()
medal = almedal.dropna()
nomedals = almedal[(almedal['Medal'].isnull())].index

In [241]:
medal['COUNTRY'] = medal.index
medal['Medal']=medal['Medal'].map(int)

In [323]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
d = pd.merge(medal,df, on='COUNTRY', how='left')
d.head(1)

,ID,Age,Height,Weight,Year,Medal,COUNTRY,GDP (BILLIONS),CODE
0,8918075,1836.0,9212.0,4020.0,247720,2,Afghanistan,21.71,AFG


In [324]:
d[(d['CODE'].isnull())]

,ID,Age,Height,Weight,Year,Medal,COUNTRY,GDP (BILLIONS),CODE
7,24872152,9859.0,62814.0,2.568100e+04,744924,40,Bahamas,NaN,NaN
51,6290837,2357.0,12038.0,4.984000e+03,187598,5,Individual Olympic Athletes,NaN,NaN
58,11119524,4291.0,28050.0,1.130000e+04,384384,3,Ivory Coast,NaN,NaN
88,53917474,17790.0,104528.0,3.754100e+04,1607570,67,North Korea,NaN,NaN
108,297169257,101882.0,668190.0,2.596930e+05,8902536,638,South Korea,NaN,NaN
122,22708176,9734.0,65147.0,2.687250e+04,785822,37,Trinidad,NaN,NaN
125,809225029,309697.0,1395588.0,5.591310e+05,24084678,2068,UK,NaN,NaN
126,1258144263,483369.0,2622879.0,1.038127e+06,37142720,5637,USA,NaN,NaN
134,19781950,7955.0,42704.0,1.761900e+04,585074,1,"Virgin Islands, US",NaN,NaN


In [326]:
d['CODE'] =  d['CODE'].fillna(d['COUNTRY'])

In [335]:
d = d.replace('Bahamas', 'BHM')
d = d.replace('Ivory Coast', 'CIV')
d = d.replace('North Korea', 'PRK')
d = d.replace('South Korea', 'KOR')
d = d.replace('Trinidad', 'BRB')
d = d.replace('UK', 'GBR')
d = d.replace('Virgin Islands, US', 'VGB')

In [338]:
import plotly.plotly as py
data = [ dict(
        type = 'choropleth',
        locations = d['CODE'],
        z = d['Medal'],
        text = d['COUNTRY'],
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            title = 'Total Number of Medals'),
      ) ]

layout = dict(
    title = 'Medal count wrt each country',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d3-world-map' )

### The Map does not contain countries with no medals

In [339]:
print("#### COUNTRIES WITH NO OLYMPIC MEDALS ###\n")
for i in nomedals:
    print(i)

#### COUNTRIES WITH NO OLYMPIC MEDALS ###

Albania
American Samoa
Andorra
Angola
Antigua
Aruba
Bangladesh
Belize
Benin
Bhutan
Boliva
Bosnia and Herzegovina
Brunei
Burkina Faso
Cambodia
Cape Verde
Cayman Islands
Central African Republic
Chad
Comoros
Cook Islands
Democratic Republic of the Congo
Dominica
El Salvador
Equatorial Guinea
Gambia
Guam
Guinea
Guinea-Bissau
Honduras
June Climene
Kiribati
Laos
Lesotho
Liberia
Libya
Madagascar
Malawi
Maldives
Mali
Malta
Marshall Islands
Mauritania
Micronesia
Myanmar
Nauru
Nicaragua
Oman
Palau
Palestine
Papua New Guinea
Refugee Olympic Athletes
Republic of Congo
Rika II
Rwanda
Saint Kitts
Saint Lucia
Saint Vincent
Samoa
San Marino
Sao Tome and Principe
Seychelles
Sierra Leone
Singapore-1
Singapore-2
Solomon Islands
Somalia
South Sudan
Swaziland
Timor-Leste
Turkmenistan
Tuvalu
Unknown
Vanuatu
Virgin Islands, British
Yemen
